## Data from CSV

In [3]:
# import pandas as pd
# import numpy as np
# from sklearn.preprocessing import StandardScaler

# # Load dataset
# data = pd.read_csv("./data/TCQ_DemandSum_DST.csv")
# data['Target'] = data['Sum of DST']

# # Hardcoded average demand, 0 means no subtraction
# target_avg = 0

# # Subtract the average from the data
# data['Target'] = data['Target'] - target_avg

# # Normalize the data using the same scaler used during training
# scaler = StandardScaler()
# data['Target'] = scaler.fit_transform(data['Target'].values.reshape(-1,1))

# # Function to create dataset with sequences of 48 demand values
# def create_dataset(dataset, weeks=0, days=0, periods=48):
#     look_back=periods + (days * 48) + (weeks * 7 * 48)
#     X, Y = [], []
#     for i in range(len(dataset)-look_back):
#         X.append(dataset[i:(i+look_back)])
#         Y.append(dataset[i + look_back])
#     return np.array(X), np.array(Y)

# # Create dataset for prediction
# weeks = 1
# days = 0
# periods = 0
# predict_X, predict_Y = create_dataset(data['Target'].values, weeks=weeks, days=days, periods=periods)

# # Reshape input to be [samples, time steps, features]
# predict_X = np.reshape(predict_X, (predict_X.shape[0], predict_X.shape[1], 1))


## Data from DB

In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import pyodbc

# Define connection parameters
hostname = "HOSTNAME"
database = "DATABASE"
table = "TABLE"
periods = 48

# Define your SQL query to fetch the last 49 periods of data
sql_query = f"""
    SELECT TOP {periods} * 
    FROM {table}
    ORDER BY DateColumn DESC;  -- Replace DateColumn with the actual column name representing the date
"""

# Establish a connection to the SQL server
connection_string = f"DRIVER={{SQL Server}};SERVER={hostname};DATABASE={database};Trusted_Connection=yes;"
connection = pyodbc.connect(connection_string)

# Execute the SQL query and fetch data into a DataFrame
data = pd.read_sql(sql_query, connection)

# Close the connection
connection.close()

# Perform data preprocessing as before
data['Target'] = data['Sum of DST']
target_avg = 0
data['Target'] = data['Target'] - target_avg
scaler = StandardScaler()
data['Target'] = scaler.fit_transform(data['Target'].values.reshape(-1,1))

# Create dataset for prediction
def create_dataset(dataset):
    return np.array(dataset)

predict_X = create_dataset(data['Target'].values)

# Reshape input to be [samples, time steps, features]
predict_X = np.reshape(predict_X, (predict_X.shape[0], predict_X.shape[1], 1))


ImportError: dlopen(/Users/matthew/Desktop/SDC_Demand_Forecast/venv/lib/python3.9/site-packages/pyodbc.cpython-39-darwin.so, 0x0002): Library not loaded: /opt/homebrew/opt/unixodbc/lib/libodbc.2.dylib
  Referenced from: <FB8F2083-C5E4-38CB-B0EC-0F787727A7D3> /Users/matthew/Desktop/SDC_Demand_Forecast/venv/lib/python3.9/site-packages/pyodbc.cpython-39-darwin.so
  Reason: tried: '/opt/homebrew/opt/unixodbc/lib/libodbc.2.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/unixodbc/lib/libodbc.2.dylib' (no such file), '/opt/homebrew/opt/unixodbc/lib/libodbc.2.dylib' (no such file)

## Predict using trained model

In [ ]:

import os
import glob
from keras.models import load_model

# Define the directory where your model files are stored
model_directory = "./model/"

# Get a list of all model files in the directory
model_files = glob.glob(os.path.join(model_directory, "lstm_*.h5"))

# Sort the list of model files by modification time (most recent first)
model_files.sort(key=os.path.getmtime, reverse=True)

# Select the most recent model file
most_recent_model_file = model_files[0]

# Load the selected model
model = load_model(most_recent_model_file)

# Print the path of the loaded model for verification
print("Loaded model:", most_recent_model_file)


# Make predictions
predict_result = model.predict(predict_X)

# Invert predictions to original scale
inverted_predictions = scaler.inverse_transform(predict_result) + target_avg

# Print or use the predictions as needed
print("Predictions:")
print(inverted_predictions)
